<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Pytorch/LearningPytorch_3_pytorch_linghtning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pytorch-lightning

     |████████████████████████████████| 307kB 13.6MB/s 
     |████████████████████████████████| 276kB 13.8MB/s 
     |████████████████████████████████| 829kB 47.7MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=920d808870e22edd253d5599c07bba5167c8bb64e1524206228377b49d58ef32
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=680709be99ab95d15bf6a899f172861e36406e3e9fdc415cfa3151380be56c50
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [3]:
# コードを更にスッキリ（可読性を上げる）できるフレームワークなPyTorch Lightning
import pytorch_lightning as pl
pl.__version__

'0.8.4'

In [8]:
# 下準備が必要です 学習データ 検証用データ テスト用データ に分けておく
# testデータはまあ 0.2くらいで
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x, t = load_iris(return_X_y=True) # data train
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)
dataset = torch.utils.data.TensorDataset(x, t)
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val
torch.manual_seed(7)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
batch_size = 10
# Dataloaderに格納、trainだけshuffle
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [9]:
class Net(pl.LightningModule):
    def __init__(self, input_size = 4, hidden_size=4, output_size=3, batch_size=10):
        super(Net, self).__init__()
        self.L1 = nn.Linear(input_size,hidden_size)
        self.L2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        self.bn = nn.BatchNorm1d(input_size)
        # https://qiita.com/cometscome_phys/items/e89f89eab52bcb1e9a3f
        # 正規化 batchnorm1d
    def forward(self, x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        return x
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
        # return nn.CrossEntropyLoss(self, y ,t)
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)
    
    @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True)
    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results


In [11]:
# def Trainer(max_epochs = 1):
#     for epoch in range(max_epochs):
#         for batch in train_loader:
#             x, t = batch
#             x = x.to(device)
#             t = t.to(device)

#             optimizer.zero_grad()
#             y = net(x)
#             # 正解率を出力?
#             y_label = torch.argmax(y, dim=1)
#             acc = torch.sum(y_label == t) * 1.0
#             acc = acc / len(t)
#             loss = criterion(y, t)
#             print("accuracy: ", acc, "loss: ", loss.item())# item => tensor.Tensor => float
#             loss.backward()
#             optimizer.step()

In [15]:
# net = Net() # 学習モデルのインスタンス化
# trainer = Trainer() # 学習用のインスタンス化と学習の設定
# trainer.fit(net) # 学習ループ実行
# 3 行で 済んでしまいます！
torch.manual_seed(0)
net = Net()
print(net.L1.weight, net.L1.bias) # 学習前のパラメータ

from pytorch_lightning import Trainer
# このTrainer ってなに...?? 定義しないとかあかん?
trainer = Trainer(max_epochs=1)

trainer.fit(net) # 学習ループ実行
print(net.L1.weight, net.L1.bias)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | L1   | Linear      | 20    
1 | L2   | Linear      | 15    
2 | bn   | BatchNorm1d | 8     


Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True) Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 2 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Parameter containing:
tensor([[ 0.2048,  0.4507, -0.4385, -0.3950],
        [-0.2307,  0.1205, -0.0504,  0.3817],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True) Parameter containing:
tensor([ 0.2470,  0.2942, -0.3390, -0.2177], requires_grad=True)


**パラメータが変化していることが確認出来るね** 

Trainer引数, 色々あるよ

max_epochs, min_epochs, show_progress_bar

train_percent_check 1.0, val, test ..　各データに対する確認の比率

early_stop_callback False, 早期終了の使用の有無

gpus , 使用するGPUの数

distributed_backend, 分散学習の方法




In [17]:
# 実装　分けてみる
# 学習用データ用クラス
class TrainNet(pl.LightningModule):
    @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True)
    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss':loss}
        return results

In [18]:
# 検証用
class ValidationNet(pl.LightningModule):
    @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)
    def validation_step(self, batch, batch_nb):
        x, t = batch
        y  =self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss':loss, 'val_acc':acc}
        return results
    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        results = {'val_loss':avg_loss, 'val_acc':avg_acc}
        return results

In [19]:
# テスト用
class TestNet(pl.LightningModule):
    @pl.data_loader
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)
    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results
    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs()]).mean()
        results = {'test_loss':avg_loss, 'test_acc':avg_acc}
        return results

In [22]:
class Net(TrainNet,ValidationNet, TestNet):
    def __init__(self, input_size=4, hidden_size=4, output_size=3,batch_size = 10):
        super(Net, self).__init__()
        self.L1 = nn.Linear(input_size, hidden_size)
        self.L2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

        self.bn = nn.BatchNorm1d(input_size)
    def forward(self, x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        return x
    def lossfun(self, y, t):
        return F.cross_entropy(y,t)
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr = 0.1)

In [23]:
# クラスを分けると
torch.manual_seed(0)
net = Net()
trainer = Trainer(max_epochs=100)
trainer.fit(net)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name | Type        | Params
-------------------------------------
0 | L1   | Linear      | 20    
1 | L2   | Linear      | 15    
2 | bn   | BatchNorm1d | 8     
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Con

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 2 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1